
## 实战项目：图像标注

---

在该 notebook 中，你要学习的是如何从 [COCO 数据集](http://cocodataset.org/#home) 中对数据进行加载和预处理。此外，你还要设计一个CNN-RNN模型，使其自动生成图像标注。
但是，**models.py**文件，从而实现你自己的CNN编码器和RNN解码器。我们将对你的**models.py** 文件进行评分。

点击以下链接，即可进入此 notebook：
- [Step 1](#step1): 了解数据加载器
- [Step 2](#step2): 使用数据加载器获取批次
- [Step 3](#step3): 使用CNN编码器进行实验
- [Step 4](#step4): 实现RNN解码器

<a id='step1'></a>
## Step 1: 了解数据加载器

我们已经编写了一个 [ 数据加载器](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader) ，你可以使用它来批量加载COCO数据集。

在下面的代码单元格中，你可以使用 **data_loader.py** 中的`get_loader` 函数对数据加载器初始化。

> 在这个项目中，请不要修改 **data_loader.py** 文件，务必保留其原样。

 `get_loader` 函数将 **data_loader.py** 中可以用来探索的许多参数作为输入。现在，花一些时间在新窗口中打开 **data_loader.py**，并研究这些参数。大多数参数必须保留其默认值，只有以下参数的值允许被修改：
1. **`transform`** -  [图像转换 ](http://pytorch.org/docs/master/torchvision/transforms.html) 具体规定了应该如何对图像进行预处理，并将它们转换为PyTorch张量，然后再将它们用作CNN编码器的输入。在这里，我们建议你保留`transform_train`中提供的转换方法。之后，你可以选择自己的图像转换方法，对COCO图像进行预处理。
2. **`mode`** - `'train'`（用于批量加载训练数据）或 `'test'`（用于测试数据），二者中的一个。我们将分别说明数据加载器处于训练模式或测试模式的情况。参照该 notebook 中的说明进行操作时，请设置`mode='train'`.`'train'`，这样可以使数据加载器处于训练模式。
3. **`batch_size`** - 它是用于确定批次的大小。训练你的模型时，它是指图像标注对的数量，用于在每个训练步骤中修改模型权重。
4. **`vocab_threshold`** - 它是指在将单词用作词汇表的一部分之前，单词必须出现在训练图像标注中的总次数。在训练图像标注中出现少于`vocab_threshold` 的单词将被认为是未知单词。
5. **`vocab_from_file`** -  它是指一个布尔运算（Boolean），用于决定是否从文件中加载词汇表。

接下来，我们将更详细地描述`vocab_threshold` 和 `vocab_from_file`参数。现在，运行下面的代码单元格。要有耐心哦，可能需要几分钟才能运行！

In [1]:
#vocab_threshold   确定为词汇表内的词最少出现的次数,否则unknown words
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
!pip install nltk
import nltk
nltk.download('punkt')
from data_loader import get_loader
from torchvision import transforms

# 定义转化器预处理训练集图片
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # 把较小的边缘缩小的256
    transforms.RandomCrop(224),                      # 随机剪裁224x224
    transforms.RandomHorizontalFlip(),               # 有0.5的概率水平翻转
    transforms.ToTensor(), 
    transforms.Normalize((0.485, 0.456, 0.406),      # 标准化图像
                         (0.229, 0.224, 0.225))])
# 设定最小的次数阈值
vocab_threshold = 5
# 批量大小
batch_size = 10
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=False)

You are using pip version 9.0.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
loading annotations into memory...
Done (t=0.83s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...
Done (t=0.82s)
creating index...


  0%|          | 623/414113 [00:00<01:06, 6227.45it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:08<00:00, 6019.74it/s]


运行上面的代码单元格时，数据加载器会存储在变量`data_loader`中。

你可以将相应的数据集以`data_loader.dataset` 的方式访问。 此数据集是**data_loader.py**中`CoCoDataset`类的一个实例。 如果你对数据加载器和数据集感到陌生，我们建议你查看 [ 此 PyTorch 教程 ](http://pytorch.org/tutorials/beginner/data_loading_tutorial.html)。

### 了解 `__getitem__` 方法

 `CoCoDataset`类中的`__getitem__`方法用于确定图像标注对在合并到批处理之前应如何进行预处理。 PyTorch中的所有`Dataset` 类都是如此。如果你此感到陌生，请查看 [上面链接中的教程 ](http://pytorch.org/tutorials/beginner/data_loading_tutorial.html)。

当数据加载器处于训练模式时，该方法将首先获得训练图像的文件名（`path`）及其对应的标注（`caption`）。

#### 图像预处理 

图像预处理相对比较简单（来自`CoCoDataset`类中的`__getitem__`方法）：

In [2]:
sample_caption = 'A person doing a trick on a rail while riding a skateboard.'

下面的代码,先将字符串小写,然后通过[`nltk.tokenize.word_tokenize`](http://www.nltk.org/) 函数,分解为单词的列表.

In [3]:
import nltk

sample_tokens = nltk.tokenize.word_tokenize(str(sample_caption).lower())
print(sample_tokens)

['a', 'person', 'doing', 'a', 'trick', 'on', 'a', 'rail', 'while', 'riding', 'a', 'skateboard', '.']


在下面代码,定义一个空的列表,然后添加一个代表着开始的整数,表示开始的字符串(`"<start>"`) 

In [4]:
sample_caption = []
start_word = data_loader.dataset.vocab.start_word
print('Special start word:', start_word)
sample_caption.append(data_loader.dataset.vocab(start_word))
print(sample_caption)

Special start word: <start>
[0]


下面代码,为每一个单词赋予整数

In [5]:
sample_caption.extend([data_loader.dataset.vocab(token) for token in sample_tokens])
print(sample_caption)

[0, 3, 98, 754, 3, 396, 39, 3, 1009, 207, 139, 3, 753, 18]


在下面代码,添加一个代表着结束的整数,表示结束的字符串(`"<start>"`) 所以整数0代表句子开始,1表示代表句子结束

In [6]:
end_word = data_loader.dataset.vocab.end_word
print('Special end word:', end_word)
sample_caption.append(data_loader.dataset.vocab(end_word))
print(sample_caption)

Special end word: <end>
[0, 3, 98, 754, 3, 396, 39, 3, 1009, 207, 139, 3, 753, 18, 1]


In [7]:
import torch

sample_caption = torch.LongTensor(sample_caption)
print(sample_caption)

tensor([    0,     3,    98,   754,     3,   396,    39,     3,  1009,
          207,   139,     3,   753,    18,     1])



[<start>, 'a', 'person', 'doing', 'a', 'trick', 'while', 'riding', 'a', 'skateboard', '.', <end>]--[0, 3, 98, 754, 3, 396, 207, 139, 3, 753, 18, 1]
    
    
单词-数字对照表

In [8]:
dict(list(data_loader.dataset.vocab.word2idx.items())[:10])

{'<start>': 0,
 '<end>': 1,
 '<unk>': 2,
 'a': 3,
 'very': 4,
 'clean': 5,
 'and': 6,
 'well': 7,
 'decorated': 8,
 'empty': 9}

可以从vocabulary.py知道, word2idx字典是通过遍历训练集数据得到的.如果一个单词的出现的次数大于于`vocab_threshold`, 然后将单词添加一个key给字典,在赋值一个相应不重复的整数. 

In [9]:
vocab_threshold = 4
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=False)

loading annotations into memory...
Done (t=0.76s)
creating index...
index created!
[0/414113] Tokenizing captions...
[100000/414113] Tokenizing captions...
[200000/414113] Tokenizing captions...
[300000/414113] Tokenizing captions...
[400000/414113] Tokenizing captions...
loading annotations into memory...


  0%|          | 1198/414113 [00:00<01:10, 5875.43it/s]

Done (t=0.77s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:08<00:00, 6045.31it/s]


In [10]:
# Print the total number of keys in the word2idx dictionary.
print('Total number of tokens in vocabulary:', len(data_loader.dataset.vocab))

Total number of tokens in vocabulary: 9955


也有一些特殊的key比如开始 (`"<start>"`) 结束(`"<end>"`)和未知值(`"<unk>"`). 将未知值的对应整数设置为2

In [11]:
unk_word = data_loader.dataset.vocab.unk_word
print('Special unknown word:', unk_word)

print('All unknown words are mapped to this integer:', data_loader.dataset.vocab(unk_word))

Special unknown word: <unk>
All unknown words are mapped to this integer: 2


可以通过一些未出现在字典中的单词，测试一下

In [12]:
print(data_loader.dataset.vocab('jfkafejw'))
print(data_loader.dataset.vocab('ieowoqjf'))

2
2


`vocab_from_file`参数，当创建词汇表时会将创建好的词汇表存入`vocab.pkl`.所以当选择为True时，你就可以从文件读取词汇表

In [13]:
data_loader = get_loader(transform=transform_train,mode='train',batch_size=batch_size,vocab_from_file=True)

Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...


  0%|          | 1180/414113 [00:00<01:14, 5549.39it/s]

Done (t=0.82s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:08<00:00, 6080.22it/s]


接下来，到了探索如何利用data_loader来获得训练数据了

<a id='step2'></a>
## Step 2: 通过使用data_loader来获得训练数据

可以通过`data_loader.dataset.caption_lengths`, 可以得到标题的长度

In [14]:
from collections import Counter
#data_loader.dataset.caption_lengths  标注的长度
counter = Counter(data_loader.dataset.caption_lengths)#返回 value(标注长度) -- count
lengths = sorted(counter.items(), key=lambda pair: pair[1], reverse=True)
for value, count in lengths:
    print('value: %2d --- count: %5d' % (value, count))

value: 10 --- count: 86334
value: 11 --- count: 79948
value:  9 --- count: 71934
value: 12 --- count: 57637
value: 13 --- count: 37645
value: 14 --- count: 22335
value:  8 --- count: 20771
value: 15 --- count: 12841
value: 16 --- count:  7729
value: 17 --- count:  4842
value: 18 --- count:  3104
value: 19 --- count:  2014
value:  7 --- count:  1597
value: 20 --- count:  1451
value: 21 --- count:   999
value: 22 --- count:   683
value: 23 --- count:   534
value: 24 --- count:   383
value: 25 --- count:   277
value: 26 --- count:   215
value: 27 --- count:   159
value: 28 --- count:   115
value: 29 --- count:    86
value: 30 --- count:    58
value: 31 --- count:    49
value: 32 --- count:    44
value: 34 --- count:    39
value: 37 --- count:    32
value: 33 --- count:    31
value: 35 --- count:    31
value: 36 --- count:    26
value: 38 --- count:    18
value: 39 --- count:    18
value: 43 --- count:    16
value: 44 --- count:    16
value: 48 --- count:    12
value: 45 --- count:    11
v

为了生成批量的训练数据，我们首先对标注长度进行采样。在采样中，抽取的特定长度的概率需要与数据集中具有该长度的标注的数量成比例。 然后，我们检索一批图像标注对的size`batch_size`，其中，所有标注都具有采样长度。 这种用于分配批次的方法与 [这篇文章 ](https://arxiv.org/pdf/1502.03044.pdf) 中的过程相匹配，并且已被证明在不降低性能的情况下具有计算上的有效性。
运行下面的代码单元格，生成一个批次。 `CoCoDataset`类中的`get_train_indices`方法首先对标注长度进行采样，然后对与训练数据点对应的`batch_size`indices进行采样，并使用该长度的标注。 这些indices存储在`indices`下方。
这些indices会提供给数据加载器，然后用于检索相应的数据点。该批次中的预处理图像和标注存储在`images`和`captions`中。

In [20]:
import numpy as np
import torch.utils.data as data
#根据标注长度,随机取样
indices = data_loader.dataset.get_train_indices()
print('sampled indices:', indices)

new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
data_loader.batch_sampler.sampler = new_sampler
    
# Obtain the batch.
images, captions = next(iter(data_loader))
    
print('images.shape:', images.shape)
print('captions.shape:', captions.shape)

sampled indices: [355982, 47267, 331808, 201321, 200311, 219195, 322965, 22511, 402469, 350564]
images.shape: torch.Size([10, 3, 224, 224])
captions.shape: torch.Size([10, 10])


每次运行上面的代码单元格时，都会对不同的标注长度进行采样，并返回不同批次的训练数据。多次运行代码单元格，尝试检验一下吧！

在接下来的一个notebook（**2_Training.ipynb**）中训练你的模型。我们会将用于生成训练批次的代码提供给你。


<a id='step3'></a>
## Step 3: 使用CNN编码器进行实验

运行下面的代码单元格，从**model.py**中导入`EncoderCNN`和`DecoderRNN`。

In [21]:
# Watch for any changes in model.py, and re-load it automatically.
% load_ext autoreload
% autoreload 2

# Import EncoderCNN and DecoderRNN. 
from model import EncoderCNN, DecoderRNN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


在下一个代码单元格中，我们定义了一个device，你将使用它将PyTorch张量移动到GPU（如果CUDA可用的话）。 在进行下一步之前，运行此代码单元格。

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())

True


运行下面的代码单元格，在`encoder`中实例化CNN编码器。

然后，该notebook的 **Step 2**中批次的预处理图像会通过编码器，且其输出会存储在`features`中。

In [23]:
#图片嵌入的长度
embed_size = 256
encoder = EncoderCNN(embed_size)#获取编码器对象
encoder.to(device)  
images = images.to(device)
features = encoder(images)#获取解码器对象
print('type(features):', type(features))
print('features.shape:', features.shape)
assert type(features)==torch.Tensor, "Encoder output needs to be a PyTorch Tensor." 
assert (features.shape[0]==batch_size) & (features.shape[1]==embed_size), "The shape of the encoder output is incorrect."

type(features): <class 'torch.Tensor'>
features.shape: torch.Size([10, 256])


编码器使用预先训练的ResNet-50架构（删除了最终的完全连接层）从一批预处理图像中提取特征。然后将输出展平为矢量，然后通过 `Linear`层，将特征向量转换为与单词向量同样大小的向量。

![Encoder](images/encoder.png)

为了试验其他架构，也可以修改 **model.py**中的编码器。 [ 使用一个不同的预训练模型架构 ](http://pytorch.org/docs/master/torchvision/models.html)。然后[ 添加批次归一化 ](http://pytorch.org/docs/master/nn.html#normalization-layers)。

`EncoderCNN`类必须将`embed_size`作为一个输入参数，这个参数也将对应于你将在 Step 4 中实现的RNN解码器输入的维度。也可以对`embed_size`的值进行调整哦。

<a id='step4'></a>
## Step 4: 实现RNN解码器

在执行下一个代码单元格之前，必须在**model.py**中的`DecoderRNN` 类中编写`__init__`和 `forward`方法。
你的解码器将会是`DecoderRNN`类的一个实例，且必须接收下列输入：
- 包含嵌入图像特征的PyTorch张量`features`（在 Step 3 中输出，当 Step 2 中的最后一批图像通过编码器时）
- 与 Step 2中最后一批标注（`captions`）相对应的PyTorch张量。

请注意，编写数据加载器的方式应该会简化你的代码。特别是，每个训练批次都包含预处理的标注，其中所有标注都具有相同的长度（`captions.shape[1]`），因此**你无需担心填充问题**。
> 我们实现 [本文](https://arxiv.org/pdf/1411.4555.pdf)中描述的解码器，也可以实现自行选择的任何一种架构，只要至少使用一个RNN层，且隐藏维度为`hidden_size`。

虽然你将使用当前存储在notebook中的最后一个批次来测试该解码器，但你的解码器应编写为接收嵌入图像特征和预处理标注的任意批次作为输入，其中所有标注具有相同的长度。

![Decoder](images/decoder.png)

 在下面的代码单元格中，`outputs`应该是一个大小为`[batch_size, captions.shape[1], vocab_size]`的PyTorch张量。这样设计输出的目的是`outputs[i,j,k]`包含模型的预测分数，而该分数表示批次中第 `i`个标注中的第`j`个token是词汇表中第`k`个token的可能性。

In [24]:
hidden_size = 512
vocab_size = len(data_loader.dataset.vocab)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)
decoder.to(device)
captions = captions.to(device)
outputs = decoder(features, captions)

print('type(outputs):', type(outputs))
print('outputs.shape:', outputs.shape)
assert type(outputs)==torch.Tensor, "Decoder output needs to be a PyTorch Tensor."
assert (outputs.shape[0]==batch_size) & (outputs.shape[1]==captions.shape[1]) & (outputs.shape[2]==vocab_size), "The shape of the decoder output is incorrect."

type(outputs): <class 'torch.Tensor'>
outputs.shape: torch.Size([10, 10, 9955])


/opt/conda/lib/python3.6/site-packages/torch/nn/modules/rnn.py:38: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
